In [1]:
# Mrinal Pradhan
# Challenge 5
# Linear Algebra Gaussian Elimination/LU Decomposition

import numpy as np

# Gaussian Elimination
def gaussian_elimination(A, b, *, tol=1e-12):
    A = np.array(A).astype(float)
    b = np.array(b).astype(float)
    n = b.shape[0]
    if A.shape != (n, n):
        raise ValueError("A must be square and match the length of b.")
    # Forward elimination
    for i in range(n):
        maxRow = i + np.argmax(np.abs(A[i:, i]))
        if abs(A[maxRow, i]) < tol:
            raise ValueError("Matrix is singular or nearly singular (zero pivot).")
        if maxRow != i:
            A[[i, maxRow]] = A[[maxRow, i]]
            b[[i, maxRow]] = b[[maxRow, i]]
        # Make rows below this one 0 in current column
        for j in range(i + 1, n):
            ratio = A[j, i] / A[i, i]
            A[j, i:] = A[j, i:] - ratio * A[i, i:]
            b[j] = b[j] - ratio * b[i]
    # Back substitution
    x = np.zeros(n).astype(float)
    for i in range(n - 1, -1, -1):
        if abs(A[i, i]) < tol:
            raise ValueError("Matrix is singular or nearly singular during back substitution.")
        x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]
    return x

# Lu Decompostion
def lu_solve(A, b, *, tol=1e-12):
    A = np.array(A).astype(float)
    b = np.array(b).astype(float)
    n = b.shape[0]
    if A.shape != (n, n):
        raise ValueError("A must be square and match the length of b.")
    U = A.copy()
    L = np.eye(n).astype(float)
    p = np.arange(n)  

    # Factorization with partial pivoting: P@A = L@U
    for k in range(n):
        # Choose pivot row
        pivotRow = k + np.argmax(np.abs(U[k:, k]))
        if abs(U[pivotRow, k]) < tol:
            raise ValueError("Matrix is singular or nearly singular (zero pivot).")
        # Swap rows in U, track permutation, and swap the already-built part of L
        if pivotRow != k:
            U[[k, pivotRow], :] = U[[pivotRow, k], :]
            p[[k, pivotRow]] = p[[pivotRow, k]]
            if k > 0:
                L[[k, pivotRow], :k] = L[[pivotRow, k], :k]
        # Eliminate below pivot
        for i in range(k + 1, n):
            L[i, k] = U[i, k] / U[k, k]
            U[i, k:] = U[i, k:] - L[i, k] * U[k, k:]
    # Apply permutation to b: Pb
    bp = b[p]
    # Forward substitution: L y = bp
    y = np.zeros(n).astype(float)
    for i in range(n):
        y[i] = bp[i] - np.dot(L[i, :i], y[:i])
        if abs(L[i, i]) < tol:
            raise ValueError("Zero diagonal in L during forward substitution.")
        y[i] /= L[i, i]
    # Back substitution: U x = y
    x = np.zeros(n).astype(float)
    for i in range(n - 1, -1, -1):
        if abs(U[i, i]) < tol:
            raise ValueError("Zero diagonal in U during back substitution.")
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]

    return x